# Data Selection View
> A table view for the DataModel class

In [ ]:
#| default_exp data.view.select

In [ ]:
#| export
import ipywidgets as widgets
from jupyter_web_app_template.data.model import DataModel
from traitlets import link, observe

In [ ]:
#| export
class DataSelectView(widgets.VBox):
    
    def __init__(self):
        super().__init__()
        
        style = {'description_width': 'initial'}
        
        SOURCES_TEXT = '''<p>
        <b>Land-Ocean Temperature Index</b>
        <a href="https://climate.nasa.gov/vital-signs/global-temperature/"
        target="_blank">Global Temperature (NASA)</a>
        ,
        <a href="https://data.giss.nasa.gov/gistemp/"
        target="_blank">GISS Surface Temperature Analysis (NASA)</a>
        </p><p>
        This site is based on data downloaded from the following site on 2020-07-14:
        <a href="https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt"  # noqa
        target="_blank">Global Mean Estimates based on Land and Ocean Data (NASA)</a>
        <br>
        
        '''
        INTRO_TEXT = '''
        <p><b>Curve Smoothing</b>
        This tool is for smoothing and selecting land-ocean temperature data for visualization and download. Start by selecting a date
        range, and then select the smoothing algorithm you want to use. Then click through to the next step, where you will change properies
        of the curve smoothing algorithm you selected and visualize the data. In the final step, you may export the data you selected.
        </p>
        '''
        SOURCES_TEXT = '''
        <p>
        <b>About Land-Ocean Temperature Data</b>
        <a href="https://climate.nasa.gov/vital-signs/global-temperature/"
        target="_blank">Global Temperature (NASA)</a>
        ,
        <a href="https://data.giss.nasa.gov/gistemp/"
        target="_blank">GISS Surface Temperature Analysis (NASA)</a>
        </p><p>
        This site is based on data downloaded from the following site on 2020-07-14:
        <a href="https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt"  # noqa
        target="_blank">Global Mean Estimates based on Land and Ocean Data (NASA)</a>
        '''

        self.intro_text = widgets.HTML(value = INTRO_TEXT)
        self.data_source_text = widgets.HTML(value = SOURCES_TEXT)
        range_layout = {'width': '400px'}
        self.date_range = widgets.IntRangeSlider(description="Date Range", style=style, layout=range_layout)
        output_layout = {'min_height': '200px'}
        self.df_output = widgets.Output(layout = output_layout)
        
        options = ['Temperature', 'EMA', 'LOWESS', 'Savitzky-Golay']
        self.smoothing_type = widgets.Dropdown(description="Smoothing Type", options = options, style=style)
        self.next_step_btn = widgets.Button(description='Proceed', button_style='info', icon='right-arrow')
        self.hbox = widgets.HBox(children = (self.smoothing_type, self.next_step_btn))

        self.children = (self.intro_text, self.date_range, self.df_output, self.data_source_text, self.hbox)

In [ ]:
dataSelectView = DataSelectView()
dataSelectView

In [ ]:
app = widgets.Tab(children = (dataSelectView, ))
app

In [ ]:
dataModel = DataModel()
dataModel

In [ ]:
#| export
class DataSelectDelegate(DataSelectView):
    
    def __init__(self, model):
        super().__init__()
        self.model = model
        
        link((self.model, 'max'), (self.date_range, 'max'))
        link((self.model, 'min'), (self.date_range, 'min'))
        link((self.model, 'smoothing_type'), (self.smoothing_type, 'value'))
        self.date_range.value = (self.model.from_year, self.model.to_year) # initialize values from model
        self.date_range.observe(self._observe_date_range, 'value') # observe changes from range widget
    
    def _observe_date_range(self, change):
        if change['old'][0] != change['new'][0]: # if change occured in the first index of the tuple
            self.model.from_year = change['new'][0]
            self.display_df()
        elif change['old'][1] != change['new'][1]: # else if change occured in the second index of the tuple
            self.model.to_year = change['new'][1]
            self.display_df()
                                 
    def display_df(self):
        self.df_output.clear_output(wait=True)
        with self.df_output: 
            display(self.model.selected)

In [ ]:
dataSelectDelegate = DataSelectDelegate(dataModel)
dataSelectDelegate.display_df()
dataSelectDelegate

In [ ]:
dataModel.smoothing_type

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()